In [1]:
import pandas as pd
import tweepy
import re

In [2]:
# function to display data of each tweet
def printtweetdata(n, ith_tweet):
    print()
    print(f"Tweet {n}:")
    print(f"Username:{ith_tweet[0]}")
    print(f"Description:{ith_tweet[1]}")
    print(f"Date/Time:{ith_tweet[2]}")
    print(f"Following Count:{ith_tweet[3]}")
    print(f"Follower Count:{ith_tweet[4]}")
    print(f"Total Tweets:{ith_tweet[5]}")
    print(f"Retweet Count:{ith_tweet[6]}")
    print(f"Tweet Text:{ith_tweet[7]}")
    print(f"Hashtags Used:{ith_tweet[8]}")

In [3]:
# function to perform data extraction by coin
def scrape_recent(coin, users, numtweet, keywords, start_date, end_date, filename):
    if keywords:
        keywords += " AND "
    if start_date:
        start_date = " since:"+start_date
    if end_date:
        end_date = " until:"+end_date
    if users:
        users_list = users.split()
        if len(users_list)>1:
            users = ' OR from:'.join(users_list)
        users = 'from:'+users
        users += " AND "
    db = pd.DataFrame(columns=['username', 'description', 'location', 'following',
                               'followers', 'totaltweets', 'retweetcount', 'text', 'hashtags'])
    query = users+keywords+coin+start_date+end_date+' lang:en AND -is:nullcast'
    print(query)
    tweets = tweepy.Cursor(api.search, q=query, lang="en", tweet_mode='extended').items(numtweet)
    list_tweets = [tweet for tweet in tweets]
    i = 1
    for tweet in list_tweets:
        username = tweet.user.screen_name
        description = tweet.user.description
        datetime = tweet.created_at
        following = tweet.user.friends_count
        followers = tweet.user.followers_count
        totaltweets = tweet.user.statuses_count
        retweetcount = tweet.retweet_count
        hashtags = tweet.entities['hashtags']
        # in case of invalid reference in retweet, except block will be executed
        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:
            text = tweet.full_text
        hashtext = list()
        for j in range(0, len(hashtags)):
            hashtext.append(hashtags[j]['text'])

        text = re.sub("@[A-Za-z0-9_]+","", text)
        text = re.sub("#[A-Za-z0-9_]+", "", text)
        ith_tweet = [username, description, datetime, following,
                     followers, totaltweets, retweetcount, text, hashtext]
        db.loc[len(db)] = ith_tweet

        printtweetdata(i, ith_tweet)
        i = i + 1
    db = db.iloc[db['text'].drop_duplicates().index,:]
    db.to_csv(filename)

In [4]:
# function to perform data extraction by coin
def scrape_by_users(users, keywords, filename):
    if users:
        users_list = users.split()
    else:
        return
    db = pd.DataFrame(columns=['username', 'description', 'location', 'following',
                               'followers', 'totaltweets', 'retweetcount', 'text', 'hashtags'])
    num = 0
    for user in users_list:
        tweets = tweepy.Cursor(api.user_timeline, screen_name = user, lang="en", tweet_mode='extended').items(1000)
        print(tweets)
        list_tweets = [tweet for tweet in tweets]
        for i, tweet in enumerate(list_tweets):
            match = False
            for word in keywords:
                if word in tweet.full_text:
                    match = True
                    break
            if not match:
                del list_tweets[i]
                continue
            username = tweet.user.screen_name
            description = tweet.user.description
            datetime = tweet.created_at
            following = tweet.user.friends_count
            followers = tweet.user.followers_count
            totaltweets = tweet.user.statuses_count
            retweetcount = tweet.retweet_count
            hashtags = tweet.entities['hashtags']
            # in case of invalid reference in retweet, except block will be executed
            try:
                text = tweet.retweeted_status.full_text
            except AttributeError:
                text = tweet.full_text
            hashtext = list()
            for j in range(0, len(hashtags)):
                hashtext.append(hashtags[j]['text'])

            text = re.sub("@[A-Za-z0-9_]+","", text)
            text = re.sub("#[A-Za-z0-9_]+", "", text)
            ith_tweet = [username, description, datetime, following,
                         followers, totaltweets, retweetcount, text, hashtext]
            db.loc[len(db)] = ith_tweet
            printtweetdata(num, ith_tweet)
            num += 1

    db.to_csv(filename)

In [5]:
# credentials for Academic Research Twitter API
consumer_key = "qBlrqapuTvDreZNzOdS8g4dnu"
consumer_secret = "jlaEzduOdHHEyR64sfJh2PJTg72kwXKR7IhPqawD6lJf3StgP2"
access_key = "1460353844395720704-UF1QRynvk3bnD4nU99KykutSrmvkRh"
access_secret = "5B7XYpSJ4qDFrLpK0GGNDxf2r66rfJVscrvwohwnyh11a"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [6]:
#Scrape by usernames and keywords (can go further than 7 days in the past)
print("Enter usernames of interest (separated by spaces, without @):")
users = input()
print("Enter keywords/hashtags for search (separated by spaces)")
keywords = input()
print("Enter Output File Name (.csv)")
filename = input()
scrape_by_users(users, keywords, filename)
print('Scraping has completed!')

Enter usernames of interest (separated by spaces, without @):
brieflycrypto cryptoquant_com
Enter keywords/hashtags for search (separated by spaces)
#bitcoin binance ethereum
Enter Output File Name (.csv)
mixed_user.csv

Tweet 0:
Username:BrieflyCrypto
Description:Your daily #Crypto news feed

• Telegram version 👉 https://t.co/ThOQOQLho1
• Created by @danclarkie in collaboration with @imBagsy
Date/Time:2022-02-04 21:00:17
Following Count:3
Follower Count:8610
Total Tweets:14687
Retweet Count:0
Tweet Text:Bots Are Overrunning  Networks Like Solana as They Hunt for Profits

Sentiment: Negative

Brief: https://t.co/zju72kUM6z
https://t.co/vwGtJc82En
Hashtags Used:['Crypto']

Tweet 1:
Username:BrieflyCrypto
Description:Your daily #Crypto news feed

• Telegram version 👉 https://t.co/ThOQOQLho1
• Created by @danclarkie in collaboration with @imBagsy
Date/Time:2022-02-04 20:50:14
Following Count:3
Follower Count:8610
Total Tweets:14687
Retweet Count:0
Tweet Text:Bitcoin Is Once Again Ahead Of

Retweet Count:1
Tweet Text:The Super Bowl is now being called ‘Crypto Bowl’ as upstart finance firms spend millions on T.V. ads

Sentiment: Positive

The Super Bowl is perhaps the "Best vehicle" for smaller, lesser-known...

Brief: https://t.co/zju72kCCSr
https://t.co/5IDta1Kghm
Hashtags Used:[]

Tweet 40:
Username:BrieflyCrypto
Description:Your daily #Crypto news feed

• Telegram version 👉 https://t.co/ThOQOQLho1
• Created by @danclarkie in collaboration with @imBagsy
Date/Time:2022-02-04 05:40:20
Following Count:3
Follower Count:8610
Total Tweets:14687
Retweet Count:0
Tweet Text:Polkadot’s on-chain treasury is now offering $18 million to cultivate technical innovation in the Polkadot network.

Sentiment: Positive

The Polkadot Pioneers Prize program is set to announce its...

Brief: https://t.co/zju72kUM6z
https://t.co/yJwPCl3Uu5
Hashtags Used:[]

Tweet 41:
Username:BrieflyCrypto
Description:Your daily #Crypto news feed

• Telegram version 👉 https://t.co/ThOQOQLho1
• Created by @danc


Tweet 76:
Username:BrieflyCrypto
Description:Your daily #Crypto news feed

• Telegram version 👉 https://t.co/ThOQOQLho1
• Created by @danclarkie in collaboration with @imBagsy
Date/Time:2022-02-03 15:20:35
Following Count:3
Follower Count:8610
Total Tweets:14687
Retweet Count:0
Tweet Text:‘South Park’ Returns With Regular Season Episodes, Skewering Matt Damon and Cryptocurrency

Sentiment: Positive

South Park returned with its first regular season episode since 2019 - and no time was...

Brief: https://t.co/zju72kUM6z
https://t.co/9UdH3zAnQu
Hashtags Used:[]

Tweet 77:
Username:BrieflyCrypto
Description:Your daily #Crypto news feed

• Telegram version 👉 https://t.co/ThOQOQLho1
• Created by @danclarkie in collaboration with @imBagsy
Date/Time:2022-02-03 15:20:19
Following Count:3
Follower Count:8610
Total Tweets:14687
Retweet Count:0
Tweet Text:Loopring CTO Steve Guo to become new CEO

Sentiment: Positive

We are incredibly excited to announce that Steve Guo has been appointed the rol

Description:Your daily #Crypto news feed

• Telegram version 👉 https://t.co/ThOQOQLho1
• Created by @danclarkie in collaboration with @imBagsy
Date/Time:2022-02-03 03:30:37
Following Count:3
Follower Count:8610
Total Tweets:14687
Retweet Count:0
Tweet Text:A Cube Made From $11.7 Million Worth of Gold Is Sitting in Central Park— It’s being used to promote $CAST Cryptocurrency

Sentiment: Positive

A cube composed of 186 kilograms of pure 24-karat gold,...

Brief: https://t.co/zju72kCCSr
https://t.co/oYj4kLh9cP
Hashtags Used:[]

Tweet 113:
Username:BrieflyCrypto
Description:Your daily #Crypto news feed

• Telegram version 👉 https://t.co/ThOQOQLho1
• Created by @danclarkie in collaboration with @imBagsy
Date/Time:2022-02-03 03:30:21
Following Count:3
Follower Count:8610
Total Tweets:14687
Retweet Count:0
Tweet Text:Thailand Has Just Axed The 15% Tax On Crypto

Sentiment: Positive

This decision to do away with imposing taxes on the profit made by crypto trading comes after strong oppositi


Tweet 150:
Username:BrieflyCrypto
Description:Your daily #Crypto news feed

• Telegram version 👉 https://t.co/ThOQOQLho1
• Created by @danclarkie in collaboration with @imBagsy
Date/Time:2022-02-02 15:20:22
Following Count:3
Follower Count:8610
Total Tweets:14687
Retweet Count:1
Tweet Text:Polkadot could become the next sponsor of FC Barcelona. Also the ninth parachain winner is Centrifuge: tokenizing traditional financial assets. Bullish.

Sentiment: Positive

Polkadot is slated with...

Brief: https://t.co/zju72kUM6z
https://t.co/PSJoeJRSBk
Hashtags Used:[]

Tweet 151:
Username:BrieflyCrypto
Description:Your daily #Crypto news feed

• Telegram version 👉 https://t.co/ThOQOQLho1
• Created by @danclarkie in collaboration with @imBagsy
Date/Time:2022-02-02 15:00:16
Following Count:3
Follower Count:8610
Total Tweets:14687
Retweet Count:0
Tweet Text:The NFT Ecosystem Is a Complete Disaster

Sentiment: Neutral

Any way you cut it: the NFT ecosystem as it stands is a disaster. Take OpenSea,

...

Tweet 1861:
Username:cryptoquant_com
Description:Actionable on-chain data, charts, alerts for investors in $BTC, $ETH, #Stablecoins | Insights: https://t.co/4soN6l653x | Alerts: https://t.co/kINun4IPOM
Date/Time:2020-12-29 16:26:07
Following Count:7
Follower Count:184120
Total Tweets:1265
Retweet Count:12
Tweet Text:Save charts. Save time. https://t.co/Lo7n8GKULn
Hashtags Used:[]

Tweet 1862:
Username:cryptoquant_com
Description:Actionable on-chain data, charts, alerts for investors in $BTC, $ETH, #Stablecoins | Insights: https://t.co/4soN6l653x | Alerts: https://t.co/kINun4IPOM
Date/Time:2020-12-29 16:10:09
Following Count:7
Follower Count:184120
Total Tweets:1265
Retweet Count:7
Tweet Text:Now we support exchange flows data for $SUSHI, $AAVE, $UNI, $LINK!

Check out exchange reserve, inflow, outflow, netflow, transactions count, and addresses count data!

https://t.co/beqGxlvOwC https://t.co/MAXsvUNHy7
Hashtags Used:[]

Tweet 1863:
Username:cryptoquant_com
Description:Actionable on-c


Tweet 1893:
Username:cryptoquant_com
Description:Actionable on-chain data, charts, alerts for investors in $BTC, $ETH, #Stablecoins | Insights: https://t.co/4soN6l653x | Alerts: https://t.co/kINun4IPOM
Date/Time:2020-12-16 15:41:29
Following Count:7
Follower Count:184120
Total Tweets:1265
Retweet Count:80
Tweet Text:Lots of people deposited stablecoins to exchanges 7 mins before breaking $20k.

Price is all about consensus. I guess the sentiment turned around to buy $BTC at that time. 

This indicator is helpful to see buying power.

Set Alert 👉 https://t.co/xjY9mvaa15 https://t.co/gV4J1N4R9g
Hashtags Used:[]

Tweet 1894:
Username:cryptoquant_com
Description:Actionable on-chain data, charts, alerts for investors in $BTC, $ETH, #Stablecoins | Insights: https://t.co/4soN6l653x | Alerts: https://t.co/kINun4IPOM
Date/Time:2020-12-16 08:56:00
Following Count:7
Follower Count:184120
Total Tweets:1265
Retweet Count:8
Tweet Text: 
$BTC inflow from all miners to all exchanges has skyrocketed. 


Tweet 1923:
Username:cryptoquant_com
Description:Actionable on-chain data, charts, alerts for investors in $BTC, $ETH, #Stablecoins | Insights: https://t.co/4soN6l653x | Alerts: https://t.co/kINun4IPOM
Date/Time:2020-12-02 18:10:28
Following Count:7
Follower Count:184120
Total Tweets:1265
Retweet Count:4
Tweet Text:Piggybacking off  - ROC for derivative exchange reserve flow &gt;= 1.5 has produced varying levels of price weakness in 2020

Now, ROC has leapt above 1.5, possibly signaling weakness for $BTC on the horizon as long investors lever at the top of the cycle https://t.co/Z160k3Ml3p
Hashtags Used:[]

Tweet 1924:
Username:cryptoquant_com
Description:Actionable on-chain data, charts, alerts for investors in $BTC, $ETH, #Stablecoins | Insights: https://t.co/4soN6l653x | Alerts: https://t.co/kINun4IPOM
Date/Time:2020-12-02 12:27:14
Following Count:7
Follower Count:184120
Total Tweets:1265
Retweet Count:37
Tweet Text:Spot reserves flat and derivatives reserves increasing.

People ar


Tweet 1954:
Username:cryptoquant_com
Description:Actionable on-chain data, charts, alerts for investors in $BTC, $ETH, #Stablecoins | Insights: https://t.co/4soN6l653x | Alerts: https://t.co/kINun4IPOM
Date/Time:2020-11-23 05:46:25
Following Count:7
Follower Count:184120
Total Tweets:1265
Retweet Count:14
Tweet Text:Total Value Staked on $ETH 2.0 skyrocketed since Nov 20 08:00 UTC.

View Chart 👉 https://t.co/FYHH2HHlZh https://t.co/lw5DEJ63wA
Hashtags Used:[]

Tweet 1955:
Username:cryptoquant_com
Description:Actionable on-chain data, charts, alerts for investors in $BTC, $ETH, #Stablecoins | Insights: https://t.co/4soN6l653x | Alerts: https://t.co/kINun4IPOM
Date/Time:2020-11-22 14:40:56
Following Count:7
Follower Count:184120
Total Tweets:1265
Retweet Count:12
Tweet Text:ETH 2.0 dashboard by :

- Phase 0 Success Rate: 45.02%
- Phase 0 Unique Validators: 7376
- Total Value Staked: 236,032 ETH
- Unique Depositors: 698 https://t.co/3vFCtwYWas
Hashtags Used:[]

Tweet 1956:
Username:crypt


Tweet 1984:
Username:cryptoquant_com
Description:Actionable on-chain data, charts, alerts for investors in $BTC, $ETH, #Stablecoins | Insights: https://t.co/4soN6l653x | Alerts: https://t.co/kINun4IPOM
Date/Time:2020-11-10 08:05:49
Following Count:7
Follower Count:184120
Total Tweets:1265
Retweet Count:67
Tweet Text:The $BTC buy-the-dip indicator says we're still in a strong buy zone.

View Chart 👇
https://t.co/NEBSp0GFbs https://t.co/38OcQtXjrU
Hashtags Used:[]

Tweet 1985:
Username:cryptoquant_com
Description:Actionable on-chain data, charts, alerts for investors in $BTC, $ETH, #Stablecoins | Insights: https://t.co/4soN6l653x | Alerts: https://t.co/kINun4IPOM
Date/Time:2020-11-09 10:58:05
Following Count:7
Follower Count:184120
Total Tweets:1265
Retweet Count:11
Tweet Text:To see short-term $BTC whale dumping risk, check out this indicator 👇
https://t.co/vlOSNGnsdK
Hashtags Used:[]

Tweet 1986:
Username:cryptoquant_com
Description:Actionable on-chain data, charts, alerts for investo

In [31]:
#Scrape by many constraints (can go only to past 7 days)
print("Enter Cashtag of Cryptocurrency to search for (ex: $BTC)")
coin = input()
print("Enter usernames of interest (separated by spaces, without @):")
users = input()
print("Enter Number of Tweets to Scrape")
numtweet = input()
print("Enter keywords/hashtags for search (separated by OR)")
keywords = input()
print("Enter start date (yyyy-mm-dd)")
start_date = input()
print("Enter end date (yyyy-mm-dd)")
end_date = input()
print("Enter Output File Name (.csv)")
filename = input()
scrape_recent(coin, users, int(numtweet), keywords, start_date, end_date, filename)
print('Scraping has completed!')

Enter Cashtag of Cryptocurrency to search for (ex: $BTC)

Enter usernames of interest (separated by spaces, without @):
brieflycrypto cryptoquant_com
Enter Number of Tweets to Scrape
1000
Enter keywords/hashtags for search (separated by OR)
#bitcoin
Enter start date (yyyy-mm-dd)

Enter end date (yyyy-mm-dd)

Enter Output File Name (.csv)
mixed_user.csv
from:brieflycrypto OR from:cryptoquant_com AND #bitcoin AND  lang:en AND -is:nullcast

Tweet 1:
Username:cryptoquant_com
Description:Actionable on-chain data, charts, alerts for investors in $BTC, $ETH, #Stablecoins | Insights: https://t.co/4soN6l653x | Alerts: https://t.co/kINun4IPOM
Date/Time:2022-02-03 09:28:18
Following Count:7
Follower Count:184039
Total Tweets:1263
Retweet Count:5
Tweet Text:Today's BTC QuickTake (2022/2/3)

📈 30-day EMA of NVT ratio reached a 10-year low.  is undervalued in terms of network activeness.

📉 Taker Buy Sell Ratio might indicate that $BTC price is going to consolidate in a broad range for the next few 